In [4]:
from PIL import Image
from PIL import ImageSequence
import imageio
import numpy as np
from tqdm import tqdm
from scipy import ndimage
# import gif2numpy
import cv2
import array2gif
import skvideo.io
import matplotlib.pyplot as plt
import sys
import numpy as np
# skvideo.setFFmpegPath(r'C:\Users\booga\anaconda3\Lib\site-packages\skvideo\io')
INF = np.iinfo(np.int64).max
video_path = r'C:\projects\VisoMotion\data\vid3.gif'
out_path = r'C:\projects\VisoMotion\data\vid2_out.gif'
# image_path = sys.argv[1]
PATCH_SIZE = 5

In [5]:
def loop_show(vid):
    FPS = 60
    QUIT_KEY = 'q'
    cv2.namedWindow("video", cv2.WINDOW_AUTOSIZE)
    while True:
        for frame in vid:
            cv2.imshow("video", frame)
            key = cv2.waitKey(1000 // FPS)
            if key == ord(QUIT_KEY):
                cv2.destroyAllWindows()
                return


def load_video(video_path):
    # return gif2numpy.convert(video_path, BGR2RGB=False)[0]
    # return plt.imread(video_path)
    # return ndimage.imread(video_path)
    vid_obj = Image.open(video_path)
    frames = np.array([np.array(frame.copy().convert('RGB').getdata(), dtype=np.uint8).reshape(
        frame.size[1], frame.size[0], 3) for frame in ImageSequence.Iterator(vid_obj)])
    return frames


def save_video(vid, out_path):
    # array2gif.write_gif(vid, out_path)
    # imageio.mimwrite(out_path, vid)
    ims = [Image.fromarray(frame) for frame in vid]
    ims[0].save(out_path, save_all=True, append_images=ims[1:])


In [6]:

def load_video(video_path):
    # return gif2numpy.convert(video_path, BGR2RGB=False)[0]
    # return plt.imread(video_path)
    # return ndimage.imread(video_path)
    vid_obj = Image.open(video_path)
    frames = np.array([np.array(frame.copy().convert('RGB').getdata(), dtype=np.uint8).reshape(
        frame.size[1], frame.size[0], 3) for frame in ImageSequence.Iterator(vid_obj)])
    return frames

vid = load_video(video_path)

In [ ]:
def lucas_kanade_optic_flow(vid):
    vid_gray = np.array([cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in vid])
    shape = vid_gray.shape
    It, Ix, Iy = np.gradient(vid_gray)
    velocity = np.zeros(list(shape) + [2])
    for t in tqdm(range(shape[0])):
        for y in range(0, shape[1], PATCH_SIZE):
            for x in range(0, shape[2], PATCH_SIZE):
                velocity[t, y:y + PATCH_SIZE, x:x + PATCH_SIZE, :] = calc_patch_velocity(
                    Ix[t, y:y + PATCH_SIZE, x:x + PATCH_SIZE],
                    Iy[t, y:y + PATCH_SIZE, x:x + PATCH_SIZE],
                    It[t, y:y + PATCH_SIZE, x:x + PATCH_SIZE])
    return velocity


def draw_arrows(vid, velocity):
    shape = vid
    for t in range(shape[0]):
        for y in range(0, shape[1], PATCH_SIZE):
            for x in range(0, shape[2], PATCH_SIZE):
                draw_arrow(vid[t, :, :], x + PATCH_SIZE // 2, y + PATCH_SIZE // 2,
                           velocity[t, y + PATCH_SIZE // 2, x + PATCH_SIZE // 2])


def draw_arrow(img, x, y, vx, vy):
    ARROW_COLOR = (0, 0, 255)
    ARROW_WIDTH = 5
    cv2.arrowedLine(img, (x, y), (x + vx), (y + vy), ARROW_COLOR, ARROW_WIDTH)


def calc_patch_velocity(Ix_patch, Iy_patch, It_patch):
    A = np.vstack([Ix_patch.flatten(), Iy_patch.flatten()]).T
    b = -It_patch.flatten()
    return least_square(A, b)


def least_square(A, b):
    try:
        A = np.matrix(A)
        b = np.matrix(b).T
        return (np.linalg.inv(A.T * A) * (A.T * b)).squeeze()
    except np.linalg.LinAlgError:
        return np.array([0, 0])

velocity = lucas_kanade_optic_flow(vid)

  0%|                                                                                                        | 0/89 [00:00<?, ?it/s]

> <ipython-input-7-4b791d17d030>(11)lucas_kanade_optic_flow()
      9                 import pdb
     10                 pdb.set_trace()
---> 11                 velocity[t, y:y + PATCH_SIZE, x:x + PATCH_SIZE, :] = calc_patch_velocity(
     12                     Ix[t, y:y + PATCH_SIZE, x:x + PATCH_SIZE],
     13                     Iy[t, y:y + PATCH_SIZE, x:x + PATCH_SIZE],

